# Kea Research Engine - Simulation Notebook

This notebook demonstrates the Kea research pipeline in a Kaggle/Colab environment.

## Setup

1. Set your `OPENROUTER_API_KEY` in the environment
2. Run all cells to simulate a research flow

In [ ]:
# Install dependencies (Colab/Kaggle)
!pip install -q httpx pydantic

In [ ]:
import os
import asyncio
from datetime import datetime

# Set your API key
os.environ["OPENROUTER_API_KEY"] = "sk-or-your-key-here"

## 1. Test LLM Provider

In [ ]:
import httpx

async def test_llm():
    api_key = os.getenv("OPENROUTER_API_KEY")
    
    async with httpx.AsyncClient(timeout=30) as client:
        response = await client.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers={"Authorization": f"Bearer {api_key}"},
            json={
                "model": "nvidia/nemotron-3-nano-30b-a3b:free",
                "messages": [{"role": "user", "content": "What is 2+2?"}],
                "max_tokens": 50,
            }
        )
        response.raise_for_status()
        data = response.json()
    
    print("LLM Response:", data["choices"][0]["message"]["content"])
    return data

await test_llm()

## 2. Simulate Fetch URL Tool

In [ ]:
async def simulate_fetch_url(url):
    async with httpx.AsyncClient(timeout=30) as client:
        response = await client.get(url, headers={"User-Agent": "Mozilla/5.0"})
        response.raise_for_status()
    
    content = response.text[:5000]
    print(f"Fetched {len(response.text)} chars from {url}")
    return content

result = await simulate_fetch_url("https://example.com")
print(result[:500])

## 3. Simulate Research Pipeline

In [ ]:
from dataclasses import dataclass, field
from typing import List, Dict

@dataclass
class ResearchState:
    query: str
    sub_queries: List[str] = field(default_factory=list)
    facts: List[Dict] = field(default_factory=list)
    sources: List[Dict] = field(default_factory=list)
    report: str = ""
    iteration: int = 0

async def router_node(state):
    print("Router: Analyzing query...")
    return state

async def planner_node(state):
    print("Planner: Decomposing query...")
    state.sub_queries = [
        f"Latest data on {state.query}",
        f"Key statistics for {state.query}",
    ]
    return state

async def researcher_node(state):
    print("Researcher: Gathering data...")
    state.facts.append({"entity": state.query, "value": "Sample data"})
    state.sources.append({"url": "https://example.com", "title": "Sample"})
    state.iteration += 1
    return state

async def synthesizer_node(state):
    print("Synthesizer: Creating report...")
    state.report = f"# Report: {state.query}\n\nFound {len(state.facts)} facts."
    return state

async def run_pipeline(query):
    print(f"Starting Research: {query}")
    state = ResearchState(query=query)
    state = await router_node(state)
    state = await planner_node(state)
    state = await researcher_node(state)
    state = await synthesizer_node(state)
    print("Research Complete!")
    return state

state = await run_pipeline("Indonesia nickel production")
print(state.report)

## 4. Next Steps

1. Add your OpenRouter API key to enable LLM-powered nodes
2. Try different queries to test the pipeline
3. Deploy with Docker using `docker-compose up`
4. Connect to the API at `http://localhost:8080/docs`